In [2]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Flatten, Dropout, Dense
from keras.models import Model

In [3]:
# Import cifar10 data
train_dat, test_dat = cifar10.load_data()
x_train, y_train = train_dat
x_test, y_test = test_dat

170498071/170498071 [==============================] - 31s 0us/step


In [4]:
# Normalize pixel data
x_train, x_test = x_train.astype('float32')/255.0, x_test.astype('float32')/255.0
x_train.shape

(50000, 32, 32, 3)

In [5]:
# One hot encode y data(categorical)
y_train_one_hot = keras.utils.to_categorical(y_train, 10)
y_test_one_hot = keras.utils.to_categorical(y_test, 10)

In [6]:
# Create the model with softmax in last layer for predictions
input_shape = x_train[0].shape
inp = keras.layers.Input(input_shape)
X = Conv2D(filters=3, kernel_size=(2,2), padding="same", activation="relu")(inp)
X = BatchNormalization()(X)
X = Conv2D(filters=5, kernel_size=(2,2), padding="same", activation="relu")(X)
X = MaxPooling2D(pool_size=(2,2))(X)
X = Conv2D(filters=5, kernel_size=(2,2), padding="same", activation="relu")(X)
X = BatchNormalization()(X)
X = MaxPooling2D(pool_size=(2,2))(X)
X = Flatten()(X)
X = Dense(200, activation="relu")(X)
X = Dropout(0.3)(X)
X = Dense(10, activation="softmax")(X)
model = Model(inputs=inp, outputs=X)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 3)         39        
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 3)        12        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 5)         65        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 5)        0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 5)         105   

In [7]:
# Compile with loss categorical crossentropy for categorical data
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Fit over 100 epochs
model.fit(x_train, y_train_one_hot, validation_data=(x_test, y_test_one_hot), epochs=100)

Epoch 1/100
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9912 - accuracy: 0.6522 - val_loss: 1.0749 - val_accuracy: 0.6226
Epoch 2/100
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9599 - accuracy: 0.6651 - val_loss: 1.0764 - val_accuracy: 0.6282
Epoch 3/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9372 - accuracy: 0.6727 - val_loss: 1.0871 - val_accuracy: 0.6285
Epoch 4/100
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9120 - accuracy: 0.6797 - val_loss: 1.0625 - val_accuracy: 0.6315
Epoch 5/100
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8983 - accuracy: 0.6819 - val_loss: 1.0696 - val_accuracy: 0.6331
Epoch 6/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.8828 - accuracy: 0.6904 - val_loss: 1.0869 - val_accuracy: 0.6313
Epoch 7/100
1563/1563 [==============================] - 10s 6ms/step - loss: 0.8601 - accuracy: 0.6988 - val_loss: 1.1712 - val

In [11]:
# Evaluate on validation data
model.evaluate(x_test, y_test_one_hot)

313/313 [==============================] - 1s 4ms/step - loss: 1.6803 - accuracy: 0.5980


[1.6802783012390137, 0.5979999899864197]

In [ ]:
# We can see that the test accuracy is ~60%, a significant drop from the ~80%
# training accuracy. The model has overfit to the training data and merits perhaps
# an architecture shift or some sort of additional regularization.